In [1]:
DATA_PATH = r'/home/luis-felipe/Uncertainty_Estimation/data'
#CORRUPTED_DATA_PATH = os.path.join(DATA_PATH,'corrupted')

PATH_MODELS = r'/home/luis-felipe/Uncertainty_Estimation/torch_models'
PATH_trainer = r'/home/luis-felipe/Uncertainty_Estimation/torch_models/trainer'

In [2]:
import os
import torch
from torch import nn
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
from collections import defaultdict
import copy

In [3]:
# Define o computador utilizado como cuda (gpu) se existir ou cpu caso contrário
print(torch.cuda.is_available())
dev = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

True


In [4]:
import NN_models as models
import NN_utils as utils
import NN_utils.train_and_eval as TE
import cifar_data

In [5]:
from uncertainty.MonteCarlo_Dropout import MonteCarloDropout
from uncertainty.ensemble import Ensemble
import uncertainty.metrics as metrics
from uncertainty import TestAugmentation as _TTA
from uncertainty.MCBN import MonteCarloBatchNormalization as MCBN

## um

In [6]:
MODEL_ARC = 'VGG_16'
DATA = 'Cifar10'
NAME = MODEL_ARC +'_' + DATA 

data = cifar_data.__dict__[DATA](data_dir = DATA_PATH)
model_class = models.__dict__[MODEL_ARC]

model_path = os.path.join(PATH_MODELS,MODEL_ARC)
weights_path = os.path.join(model_path,DATA)

Files already downloaded and verified
Files already downloaded and verified


In [7]:
files = [f for f in os.listdir(weights_path) if os.path.isfile(os.path.join(weights_path, f)) and f.endswith(".pt")]
files = sorted(files)

In [8]:
models_dict = defaultdict()
for s in files:
    print(s)
    models_dict[s.replace('.pt','')] = model_class(num_classes = data.n_classes).to(dev)
    state_dict = torch.load(os.path.join(weights_path,s))
    models_dict[s.replace('.pt','')].load_state_dict(state_dict)
    models_dict[s.replace('.pt','')].eval()
    models_dict[s.replace('.pt','')].softmax = True

VGG16_10_ensemble_0.pt
VGG16_10_ensemble_1.pt
VGG16_10_ensemble_2.pt
VGG16_10_ensemble_3.pt
VGG_16_10_ensemble_4_checkpoint.pt
VGG_16_10_ensemble_5_checkpoint.pt
VGG_16_10_ensemble_6_checkpoint.pt
VGG_16_10_ensemble_7_checkpoint.pt


In [9]:
MAIN_MODEL_AS_MAX_ACC = 0
max_acc = 0
for n,m in models_dict.items():
    print(n)
    acc = TE.model_acc(m,data.test_dataloader)
    print(f'Acurácia = {acc}')
    if MAIN_MODEL_AS_MAX_ACC and acc > max_acc:
        main_model = m
        max_acc = acc
else:
    main_model = models_dict[files[1].replace('.pt','')]

VGG16_10_ensemble_0


/home/luis-felipe/anaconda3/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Acurácia = 93.36
VGG16_10_ensemble_1
Acurácia = 93.26
VGG16_10_ensemble_2


KeyboardInterrupt: 

In [ ]:
model = Ensemble(models_dict).to(dev)
print(f'Ensemble Total Accuracy: {TE.model_acc(model,data.test_dataloader)}')

In [ ]:
RC = metrics.selective_metrics(model,data.test_dataloader, c_list = np.arange(0.05,1.05,0.05))
RC.plot_RC(aurc = True)

In [ ]:
RC.correlation()

In [ ]:
plt.plot(RC.d_uncs['MCP'].cpu().numpy(),RC.d_uncs['MI'].cpu().numpy(),'o')

## todos

In [ ]:
for N_ENS in range(1,8):
    model = Ensemble(utils.slice_dict(models_dict,range(N_ENS))).to(dev)
    
    TE.model_acc(model,data.test_dataloader, set_eval = True)
    RC = metrics.selective_metrics(model,data.test_dataloader, c_list = np.arange(0.05,1.05,0.05))
    RC.plot_RC(aurc = True)
    plt.title(f'M = {N_ENS}')
    plt.show()


## todos2

In [ ]:
uncs = ['MCP']#, 'MI']
plt.figure(figsize=RC.FIGSIZE, dpi=80)
for N_ENS in range(1,8):
    model = Ensemble(utils.slice_dict(models_dict,range(N_ENS))).to(dev)
    TE.model_acc(model,data.test_dataloader, set_eval = True)
    RC = metrics.selective_metrics(model,data.test_dataloader, c_list = np.arange(0.05,1.05,0.05))
    d_uncs_ = copy.copy(RC.d_uncs)
    RC.d_uncs = utils.slice_dict(d_uncs_,uncs)
    risks = RC.RC_curves()
    for name,risk in risks.items():
        label = N_ENS
        plt.plot(RC.c_list,risk, label = label, linewidth = RC.LINEWIDTH,linestyle = next(RC.linecycler))
    
#plt.title(f'M = {N_ENS}')
plt.grid()
plt.legend()
plt.show()

In [ ]:
uncs = ['MI']#, 'MI']
plt.figure(figsize=RC.FIGSIZE, dpi=80)
for N_ENS in range(1,8):
    model = Ensemble(utils.slice_dict(models_dict,range(N_ENS))).to(dev)
    TE.model_acc(model,data.test_dataloader, set_eval = True)
    RC = metrics.selective_metrics(model,data.test_dataloader, c_list = np.arange(0.05,1.05,0.05))
    d_uncs_ = copy.copy(RC.d_uncs)
    RC.d_uncs = utils.slice_dict(d_uncs_,uncs)
    risks = RC.RC_curves()
    for name,risk in risks.items():
        label = N_ENS
        plt.plot(RC.c_list,risk, label = label, linewidth = RC.LINEWIDTH,linestyle = next(RC.linecycler))
    
#plt.title(f'M = {N_ENS}')
plt.grid()
plt.legend()
plt.show()